Some examples from the Spark Summit class where the differecne between reduceByKey and groupByKey operations illustrate the subtle difference.
For small datasets this may not be a sigficant performance and network traffic hit, but for large sets, you do want to use reduceByKey operation.

Also, good illustration of the difference between map() and flatMap() transformations and how to use them against different types of RDDs

Parallelize some numbers across a few partitions, in this case let the default be 2, I believe

In [3]:
baseRDD = sc.parallelize(range(10))
baseRDD.collect()

Use lambda function first to create a tuple of three elements, and then use flatmap on it to create a list of tranformed values.

In [5]:
transRDD = baseRDD.flatMap(lambda n: (n, n*100, 42))

we have two rdds, one the base, and the second transformed in value, with its tuples flattend out by using flatMap() operations

In [7]:
print (baseRDD.collect())
print (transRDD.collect())

create a triples RDD with tuples of 3 elements.

In [9]:
triplesRDD = baseRDD.map(lambda n: (n, n*100, 42))
triplesRDD.collect()



Using flatMap we create yet another RDD which is now flattaned.

In [11]:
flatTriplesRDD = triplesRDD.flatMap(lambda n: n)

In [12]:
print (baseRDD.collect())
print (transRDD.collect())
print (flatTriplesRDD.collect())

An illustration of how to filter out unwanted elements

In [14]:
nonZeroRDD = flatTriplesRDD.filter(lambda n: n > 0)
nonZeroRDD.collect()


In [15]:
sorted = nonZeroRDD.collect()


In [16]:
sorted


The list is now sorted using Python method on the list.sor()

In [18]:
sorted.sort()

As we can see, since this is a Python notebook and sorted is at list, we can employ its sort() method, which transforms the list in place

In [20]:
print sorted

GroupBy allows us the take an existing RDD and create another one arranged or groupedBy a key of our liking. In this case, we can use the first letter of the name.
Create an RDD as a list of names.

In [22]:
namesRDD = sc.parallelize(["Jules", "James", "John", "Adam", "Mary", "Jane", "Anna", "Fred", "Bruce", "Hasan", "Felicia", "Mona", "Mary", "Achim", "Angelica", "Mike", "Kirk", "Karl", "Ryan", "Jim", "Kay", "Marie", "Vista", "Victor", "Xaviar", "Crista"])

transfrom them by grouping them into (Key, Value), where Key is the first letter of the name, while value is the list of names that begin with the letter Key.

In [24]:
namesByGroupRDD = namesRDD.groupBy(lambda n: n[0])

In [25]:
type(namesByGroupRDD)

Python way to print them out.

In [27]:
print [(k , list(v)) for (k, v) in namesByGroupRDD.collect()]

another example where we have an existing RDD that is already a (K,V) format.

In [29]:
pairsRDD = sc.parallelize([('A', 4), ('C', 1), ('B', 1), ('F', 2), ('H', 1), ('K', 2), ('J', 5), ('A', 3), ('B', 2)])
                      

As before, we group them by key, so all values for a key are group together as a list. For example, (A, [4, 3])

In [31]:
pairsByKeyRDD = pairsRDD.groupByKey()
print (list((j[0], list(j[1])) for j in pairsByKeyRDD.collect()))


A simple way to demonstrate the succintness of WordCount in Spark. Amazingly is small amount of code!

create an word RDD.

In [34]:
words = sc.parallelize(["one", "two", "two", "three", "three", "three"])

Transform the RDD into (key, value) pair

In [36]:
wordsPairs = words.map(lambda w: (w, 1))

Use reduceByKey() transformation to add the count

In [38]:
wordsCount = wordsPairs.reduceByKey(lambda a, b: a + b)

In [39]:
wordsCount.collect()

Another less efficient way to to use groupByKey() transformation. Note that the former is a lot effecient and results in less network traffic during shuffle stage

In [41]:
wordsCountByGroup = wordsPairs.groupByKey().map(lambda t: (t[0], sum(t[1])))

As you can see the results are identical

In [43]:
wordsCountByGroup.collect()